<a href="https://colab.research.google.com/github/Henrique-soliveira/github-slideshow/blob/Henrique-soliveira-patch-1/Estatistica_aula_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [7]:
! head -5 'dados_1997_2011_paises_csv.csv'

pais;ano;idh;corrupcao_indice;competitividade_indice;globalizacao_indice;pib;populacao
�frica do Sul;1997;0,6328;56;44,54;52,60298;148814151;43353,632
�frica do Sul;1998;0,6272;57;31,11;54,51911;134295557;43961,924
�frica do Sul;1999;0,6216;50;43,9;61,04379;133183581;44526,272
�frica do Sul;2000;0,6160;52;51,52;62,47182;132877648;45064,098


In [8]:
dados = pd.read_csv('dados_1997_2011_paises_csv.csv',
                    encoding = 'latin1',
                    sep = ';',
                    decimal = ',')
dados

,pais,ano,idh,corrupcao_indice,competitividade_indice,globalizacao_indice,pib,populacao
0,África do Sul,1997,0.6328,56,44.54,52.60298,1.488142e+08,43353.63200
1,África do Sul,1998,0.6272,57,31.11,54.51911,1.342956e+08,43961.92400
2,África do Sul,1999,0.6216,50,43.90,61.04379,1.331836e+08,44526.27200
3,África do Sul,2000,0.6160,52,51.52,62.47182,1.328776e+08,45064.09800
4,África do Sul,2001,0.6126,50,45.12,62.04144,1.184790e+08,45576.21500
...,...,...,...,...,...,...,...,...
280,Reino Unido,2007,0.8560,86,75.45,82.12010,1.448770e+06,61337.36992
281,Reino Unido,2008,0.8600,86,71.90,81.67998,1.447825e+06,61731.27576
282,Reino Unido,2009,0.8600,84,76.07,79.31000,1.377247e+06,62086.03336
283,Reino Unido,2010,0.8620,77,76.81,80.18000,1.395893e+06,62438.13815


In [ ]:
dados.groupby('pais') \
            .agg(min_idh = pd.NamedAgg('idh', 'min'),
                 max_idh = pd.NamedAgg('idh', 'mean'),
                 media_idh = pd.NamedAgg('idh', 'max')) \
            .reset_index() \
            .sort_values('media_idh', ascending = False)

## Moda

In [ ]:
# fazemos assim
dados.groupby('pais')['idh'].apply(lambda x: x.mode()[0]).to_frame('moda_idh').reset_index()

## Mediana


In [ ]:
dados.groupby('pais') \
     .agg(min_idh = pd.NamedAgg('idh', 'min'),
          media_idh = pd.NamedAgg('idh', 'mean'),
          mediana_idh = pd.NamedAgg('idh', 'median'),         
          max_idh = pd.NamedAgg('idh', 'max')) \
     .reset_index() \
     .sort_values('media_idh', ascending = False)

# Percentis

In [37]:
# Não funciona
dados.groupby('pais') \
     .agg(min_idh = pd.NamedAgg('idh', 'min'),
          media_idh = pd.NamedAgg('idh', 'mean'),
          mediana_idh = pd.NamedAgg('idh', 'median'),
          percentil_idh_25 = pd.NamedAgg('idh', percentil_25),       
          max_idh = pd.NamedAgg('idh', 'max')) \
     .reset_index() \
     .sort_values('media_idh', ascending = False)

NameError: ignored

In [ ]:
#estratégia 01
dados.groupby('pais')['idh'] \
     .apply(lambda x: x.quantile([0.55, 0.85])) \
     .to_frame() \
     .reset_index() \
     .rename(columns = {'level_1':'percentil'})

In [50]:
#estratéia 02
def percentil_25(x):
  return np.quantile(x, 0.25)

In [ ]:
dados.groupby('pais') \
     .agg(min_idh = pd.NamedAgg('idh', 'min'),
          media_idh = pd.NamedAgg('idh', 'mean'),
          mediana_idh = pd.NamedAgg('idh', 'median'),
          percentil_idh_25 = pd.NamedAgg('idh', percentil_25),       
          max_idh = pd.NamedAgg('idh', 'max')) \
     .reset_index() \
     .sort_values('media_idh', ascending = False)

## Variabilidade

In [53]:
media_paises = dados.groupby('pais') \
                           .agg(media_idh = pd.NamedAgg('idh', 'mean')) \
                           .reset_index()

In [ ]:
media_paises

In [55]:
dados = dados.merge(media_paises, how = 'left', on = 'pais')


In [ ]:
dados.head(3)

In [57]:
dados['desvio'] = dados['idh'] - dados['media_idh']

In [ ]:
dados.head(3)

In [ ]:
from pandas.core.groupby import NamedAgg
from numpy.core.fromnumeric import var
# desvio medio absoluto, variacia, desvio padrão
dados.groupby('pais') \
     .agg(desvio_abs_idh = pd.NamedAgg ('idh', 'mad'),
          var_idh = pd.NamedAgg('idh', 'var'), 
          dp_idh = pd.NamedAgg('idh', 'std'))